In [2]:
import torch


# Example usage
N, T, B = 2, 5, 16
in_features, out_features = B//4, 10

# create input tensor of shape [NxTxB/4]
x = torch.randn(N, T, in_features)

x = x.view(-1, x.shape[-1])


In [2]:
import torch

outputs = [{'test_frame_recall': torch.tensor(0.3688), 'test_doa_error': torch.tensor(float('nan'))}]
num_subsets = len(outputs)

for subset_idx in range(num_subsets):
    frame_recall = torch.stack([x['test_frame_recall'] for x in outputs[subset_idx]])
    doa_error = torch.stack([x['test_doa_error'] for x in outputs[subset_idx]])

TypeError: string indices must be integers

In [6]:
torch.stack([x['test_frame_recall'] for x in outputs])

tensor([0.3688])

In [4]:

# apply linear layer to each time step
x = torch.nn.Linear(in_features, out_features)(x)


In [5]:
x.shape

torch.Size([10, 10])

In [6]:

# reshape output tensor to [N x T x S]
x = x.view(*x.shape[:-1], -1)

In [7]:
x.shape

torch.Size([10, 10])

In [8]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class TimeDistributed(nn.Module):
    def __init__(self, in_features, out_features):
        super(TimeDistributed, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        # reshape input tensor to [N*T x B/4]
        x = x.view(-1, x.shape[-1])

        # apply linear layer to each time step
        x = self.linear(x)

        # reshape output tensor to [N x T x S]
        x = x.view(*x.shape[:-1], -1)

        return x


# Example usage
N, T, B = 2, 5, 16
in_features, out_features = B//4, 10

# create input tensor of shape [NxTxB/4]
x = torch.randn(N, T, in_features)

# pack input tensor and apply time-distributed linear layer
packed_x = rnn_utils.pack_padded_sequence(x, lengths=[T]*N, batch_first=True, enforce_sorted=False)
linear_layer = TimeDistributed(in_features, out_features)
y, _ = rnn_utils.pad_packed_sequence(linear_layer(packed_x)[0], batch_first=True)

# output tensor of shape [N x T x S]
print(y.shape)  # should be [2, 5, 10]


AttributeError: 'PackedSequence' object has no attribute 'view'

In [2]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class TimeDistributedFC(nn.Module):
    """
    Class for applying time distributed fully connected layers with input tensors 
    of shape  [N, T, in_features] where N is the batch_size, T is the number of time step,
    and in_features is the number of input features at each time step. 
    Output tensors will have shape [N, T, out_features], where out_features is the number 
    of output features at each time step.
    """
    def __init__(self, in_features, out_features):
        super(TimeDistributedFC, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        
        N,T = x.shape[0],x.shape[1]
        
        # reshape input tensor to [N*T x B/4]
        x = x.view(-1, x.shape[-1])

        # # apply linear layer to each time step
        # x = self.linear(x)

        # reshape output tensor to [N x T x S]
        x = x.view(N, T,-1)

        return x

In [3]:
# Example usage
N, T, B = 2, 5, 16
in_features, out_features = B//4, 10

# create input tensor of shape [NxTxB/4]
x = torch.randn(N, T, in_features)

# # pack input tensor and apply time-distributed linear layer
# packed_x = rnn_utils.pack_padded_sequence(x, lengths=[T]*N, batch_first=True, enforce_sorted=False)
linear_layer = TimeDistributedFC(in_features, out_features)
# y, _ = rnn_utils.pad_packed_sequence(linear_layer(packed_x)[0], batch_first=True)
y = linear_layer(x)

In [9]:
x[i,j,k]

tensor([ 0.3244,  0.4101, -1.5298,  0.9352])

In [4]:
import torch

N = 2
T = 20
directions_of_arrival = []
num_hypothesis = 10
        
for i in range(num_hypothesis) :
    directions_of_arrival.append(torch.randn(size = (N,T,2))) # Size [NxTx2]
    
hyp_stacked = torch.stack(directions_of_arrival, dim=-2)

In [17]:
N = 3
T = 20
directions_of_arrival = []
num_hypothesis = 10

directions_of_arrival = torch.randn(size = (N,T,2*num_hypothesis)) # Size [NxTx(2*num_hypothesis)]
hyps_splitted = torch.split(directions_of_arrival, [2 for i in range(num_hypothesis)], 2) #num_hypothesis-uples of elements of shape [N,T,2]
hyps_stacked = torch.stack([h for h in hyps_splitted], dim=2) #Tuples of elements of shape [N,T,num_hypothesis,2]

In [18]:
len(hyps_splitted)
hyps_splitted[0].shape
hyps_stacked.shape


torch.Size([3, 20, 10, 2])

In [2]:
import torch

source_activity_target_t = torch.tensor([[1, 0, 1], [0, 1, 0]])
direction_of_arrival_target_t = torch.tensor(
    [[[1, 2], [3, 4], [5, 6]], [[7, 8], [9, 10], [11, 12]]]
)

direction_of_arrival_target_t[source_activity_target_t == 0, :] = 1000

In [3]:
direction_of_arrival_target_t

tensor([[[   1,    2],
         [1000, 1000],
         [   5,    6]],

        [[1000, 1000],
         [   9,   10],
         [1000, 1000]]])

In [7]:
outputs = [{'test_frame_recall': torch.tensor(0.9925), 'test_doa_error': torch.tensor(16.2175)}, 
 {'test_frame_recall': torch.tensor(0.9950), 'test_doa_error': torch.tensor(10.0907)}, 
 {'test_frame_recall': torch.tensor(0.9950), 'test_doa_error': torch.tensor(14.7058)}, 
 {'test_frame_recall': torch.tensor(0.9862), 'test_doa_error': torch.tensor(8.9629)}, 
 {'test_frame_recall': torch.tensor(0.9613), 'test_doa_error': torch.tensor(16.7399)}, 
 {'test_frame_recall': torch.tensor(0.9975), 'test_doa_error': torch.tensor(16.7870)}, 
 {'test_frame_recall': torch.tensor(0.9775), 'test_doa_error': torch.tensor(10.8708)}, 
 {'test_frame_recall': torch.tensor(0.9962), 'test_doa_error': torch.tensor(12.1684)}, 
 {'test_frame_recall': torch.tensor(0.9912), 'test_doa_error': torch.tensor(14.0261)}, 
 {'test_frame_recall': torch.tensor(0.9900), 'test_doa_error': torch.tensor(12.9945)}]

In [12]:
type(outputs[0])

dict

In [9]:
results = {
            'model': [], 'dataset': [], 'fold_idx': [], 'subset_idx': [], 'frame_recall': [], 'doa_error': []
        }
frame_recall = torch.stack([x['test_frame_recall'] for x in outputs]).detach().cpu().numpy()
doa_error = torch.stack([x['test_doa_error'] for x in outputs]).detach().cpu().numpy()

num_sequences = len(frame_recall)

for seq_idx in range(num_sequences):
    results['model'].append('name')
    results['dataset'].append('data')
    results['fold_idx'].append(0)
    results['subset_idx'].append(0)
    results['frame_recall'].append(frame_recall[seq_idx])
    results['doa_error'].append(doa_error[seq_idx])

In [11]:
results['frame_recall']

[0.9925, 0.995, 0.995, 0.9862, 0.9613, 0.9975, 0.9775, 0.9962, 0.9912, 0.99]

In [14]:
import time
import warnings
from importlib.util import find_spec
from pathlib import Path
from typing import Any, Callable, Dict, List
from pytorch_lightning import Callback
from pytorch_lightning.loggers import Logger
from pytorch_lightning.utilities import rank_zero_only
from itertools import permutations
import numpy as np
import torch
from torch.nn.modules.loss import _Loss
import torch.nn.functional as F
from typing import Tuple

In [35]:
def compute_spherical_distance(y_pred: torch.Tensor,
                                   y_true: torch.Tensor) -> torch.Tensor:
        if (y_pred.shape[-1] != 2) or (y_true.shape[-1] != 2):
            assert RuntimeError('Input tensors require a dimension of two.')

        sine_term = torch.sin(y_pred[:, 0]) * torch.sin(y_true[:, 0])
        cosine_term = torch.cos(y_pred[:, 0]) * torch.cos(y_true[:, 0]) * torch.cos(y_true[:, 1] - y_pred[:, 1])

        return torch.acos(F.hardtanh(sine_term + cosine_term, min_val=-1, max_val=1))
    
batch = 2
num_hyps = 10
gts = torch.randn(batch,num_hyps,2)
hyps_stacked_t=torch.randn(batch,num_hyps,2)

In [34]:
time_before = time.time()
hyps_stacked_t = hyps_stacked_t.view(-1,2)
gts = gts.view(-1,2)
diff = compute_spherical_distance(hyps_stacked_t,gts)
diff = diff.view(batch,num_hyps)
spatial_epes = diff.unsqueeze(2).unsqueeze(3).unsqueeze(4)
print(time.time()-time_before)

0.0022678375244140625


In [36]:
# V2
# hyps_stacked_t of shape (batch,num_hyps,2), gts of shape (batch, num_hyps, 2)
# Compute the diff tensor using tensor operations
time_before = time.time()
sine_term = torch.sin(hyps_stacked_t[:, :, 0]) * torch.sin(gts[:, :, 0])  # (batch, num_hyps)
cosine_term = torch.cos(hyps_stacked_t[:, :, 0]) * torch.cos(gts[:, :, 0]) * torch.cos(gts[:, :, 1] - hyps_stacked_t[:, :, 1])  # (batch, num_hyps)
diff2 = torch.acos(torch.clamp(sine_term + cosine_term, min=-1, max=1))  # (batch, num_hyps)
spatial_epes2 = diff2.unsqueeze(2).unsqueeze(3).unsqueeze(4)  # (batch, num_hyps, 1, 1, 1) 
print(time.time()-time_before)

0.0016078948974609375


In [21]:
spatial_epes == spatial_epes2

tensor([[[[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]]],



        [[[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]],


         [[[True]]]]])

In [38]:
dataset_path = '/root/workspace/lightning-hydra-template/data/ansim'
dataset_path.split('/')[-1]

'ansim'

In [5]:
import torch

# define the input tensor
batch = 3
max_sources = 4
num_hyp = 5
T = torch.randn(batch, max_sources, 2)

T2 = T.unsqueeze(-1)

# # duplicate the components along the fourth dimension
# T2 = T.unsqueeze(2).repeat(1, 1, num_hyp, 1)

# # check the shapes
# print(T.shape)   # should be [batch, max_sources, 2]
# print(T2.shape)  # should be [batch, max_sources, num_hyp, 2]

# # test the indexing condition
# i = 1
# j = 2
# print(torch.all(T[:, i, :] == T2[:, i, j, :]))  # should be True


In [7]:
import torch

# define the input tensor
batch = 3
max_sources = 4
num_hyp = 5
T = torch.randn(batch, max_sources, 2)

# duplicate the components along the fourth dimension
T2 = T.unsqueeze(2).repeat(1, 1, num_hyp, 1)

# check the shapes
print(T.shape)   # should be [batch, max_sources, 2]
print(T2.shape)  # should be [batch, max_sources, num_hyp, 2]

# test the indexing condition
i = 1
j = 2
print(torch.all(T[:, i, :] == T2[:, i, j, :]))  # should be True


torch.Size([3, 4, 2])
torch.Size([3, 4, 5, 2])
tensor(True)


In [97]:
def draft_make_sampling_loss_ambiguous_gts(hyps_stacked_t, source_activity_target_t, direction_of_arrival_target_t, mode='epe'):
        # hyps_stacked_t of shape [batch,self.num_hypothesis,2]
        # source_activity_target_t of shape [batch,Max_sources]
        # direction_of_arrival_target_t of shape [batch,Max_sources,2]
        
        filling_value = 10000 #Large number (on purpose) ; computational trick to not considers the "fake" ground truths.
        # whenever the sources are not active, as the source_activity is not to be deduced by the model is these settings. 
        num_hyps = hyps_stacked_t.shape[1]
        batch = source_activity_target_t.shape[0]
        Max_sources = source_activity_target_t.shape[1]
        
        #1st padding related to the inactive sources, not considered in the error calculation (with high error values)
        mask_active_sources = source_activity_target_t == 0
        mask_active_sources = mask_active_sources.unsqueeze(-1).expand_as(direction_of_arrival_target_t)
        direction_of_arrival_target_t[mask_active_sources] = filling_value #Shape [batch,Max_sources,2]
        
        #The ground truth tensor created is of shape [batch,Max_sources,num_hypothesis,2], such that each of the 
        # tensors gts[batch,i,num_hypothesis,2] contains duplicates of direction_of_arrival_target_t along the num_hypothesis
        # dimension. Note that for some values of i, gts[batch,i,num_hypothesis,2] may contain inactive sources, and therefore 
        # gts[batch,i,j,2] will be filled with filling_value (defined above) for each j in the hypothesis dimension.
        gts =  direction_of_arrival_target_t.unsqueeze(2).repeat(1,1,num_hyps,1) #Shape [batch,Max_sources,num_hypothesis,2]
        
        #We duplicate the hyps_stacked with a new dimension of shape Max_sources
        hyps_stacked_t_duplicated = hyps_stacked_t.unsqueeze(1).repeat(1,Max_sources,1,1) #Shape [batch,Max_sources,num_hypothesis,2]

        epsilon = 0.05
        eps = 0.001
        
        #### With euclidean distance
        diff = torch.square(hyps_stacked_t_duplicated-gts) #Shape [batch,Max_sources,num_hypothesis,2]
        channels_sum = torch.sum(diff, dim=2) #Sum over the two dimensions (azimuth and elevation here). Shape [batch,Max_sources,num_hypothesis]
        spatial_epes = torch.sqrt(channels_sum + eps)  #Distance matrix [batch,Max_sources,num_hypothesis]
        
        sum_losses = torch.tensor(0.0)

        if mode == 'epe': 
            
            spatial_epe, idx_selected = torch.min(spatial_epes, dim=2) #spatial_epe of shape [batch,Max_sources]
            mask = spatial_epe >= filling_value/2 #We create a mask for only selecting the actives sources, i.e. those which were not filled with
            # direction_of_arrival_target_t[:, source_activity_target_t == 0, :] = filling_value above. 
            spatial_epe = spatial_epe*mask #[batch,Max_sources], we select only the active sources. 
            count_non_zero = torch.sum(mask!=0) #We count the number of actives sources for the computation of the mean (below). 
            loss = torch.sum(spatial_epe)/count_non_zero #We compute the mean of the diff. 
            sum_losses = torch.add(loss, sum_losses)  
            
        elif mode == 'epe-relaxed':
        
            #We compute the loss for the "best" hypothesis. 
            
            spatial_epe, idx_selected = torch.min(spatial_epes, dim=2) #spatial_epe of shape [batch,Max_sources], idx_selected of shape [batch].
            mask = spatial_epe >= filling_value/2 #We create a mask for only selecting the actives sources, i.e. those which were not filled with
            # direction_of_arrival_target_t[:, source_activity_target_t == 0, :] = filling_value above. 
            spatial_epe = spatial_epe*mask #Shape [batch,Max_sources] ; we select only the active sources. 
            count_non_zero = torch.sum(mask!=0) #We count the number of actives sources as a sum over the batch for the computation of the mean (below).
            loss0 = torch.multiply(torch.sum(spatial_epe)/count_non_zero, 1 - epsilon) #Scalar (average with coefficient)

            #We then the find the other hypothesis, and compute the epsilon weighted loss for them
            
            # At first, we remove hypothesis corresponding to "fake" ground-truth.         
            large_mask = spatial_epes >= filling_value/2 # We remove entries corresponding to "fake"/filled ground truth in the tensor spatial_epes on
            # which the min operator was not already applied. Shape [batch,Max_sources,num_hypothesis]
            spatial_epes = spatial_epes*large_mask # Shape [batch,Max_sources,num_hypothesis].
            
            # We then remove the hypothesis selected above (with minimum dist)
            mask_selected = torch.zeros_like(spatial_epes) #Shape [batch,Max_sources,num_hypothesis]
            mask_selected.scatter_(2, idx_selected.unsqueeze(-1), 1) #Shape [batch,Max_sources,num_hypothesis]
            mask_selected = ~mask_selected #Shape [batch,Max_sources,num_hypothesis], we keep only the hypothesis which are not the minimum.
            spatial_epes = spatial_epes * mask_selected #Shape [batch,Max_sources,num_hypothesis]
            
            # mask_active_sources = mask_active_sources.unsqueeze(-1).expand_as(direction_of_arrival_target_t)
            # direction_of_arrival_target_t[mask_active_sources] = filling_value #Shape [batch,Max_sources,2]
            
            # Finally, we compute the loss
            count_non_zeros = torch.sum(spatial_epes!=0)
            if count_non_zeros > 0 :
                loss = torch.multiply(torch.sum(spatial_epes)/count_non_zero, epsilon) #Scalar for each hyp
            
            sum_losses = torch.add(loss, sum_losses)
            sum_losses = torch.add(loss0, sum_losses)

        return sum_losses

torch.Size([5, 3])

In [94]:
batch = 5
Max_sources = 3
num_hypothesis = 2

spatial_epes = torch.randn(batch,Max_sources,num_hypothesis)
spatial_epe, idx_selected = torch.min(spatial_epes, dim=2) 

# print(idx_selected.shape)
mask_selected = torch.zeros_like(spatial_epes,dtype=bool)
mask_selected.scatter_(2, idx_selected.unsqueeze(-1), 1)
mask_selected = ~mask_selected

spatial_epes = spatial_epes * mask_selected

In [96]:
~mask_selected

tensor([[[ True, False],
         [ True, False],
         [False,  True]],

        [[False,  True],
         [False,  True],
         [False,  True]],

        [[False,  True],
         [ True, False],
         [ True, False]],

        [[False,  True],
         [ True, False],
         [False,  True]],

        [[ True, False],
         [False,  True],
         [False,  True]]])

In [90]:
print(spatial_epes[0,0,:])
print(mask_selected[0,0,:])

tensor([ 0.4117, -0.6108])
tensor([1., 0.])


In [79]:
import torch

batch = 5
Max_sources = 3
num_hypothesis = 2

# create the input tensor
spatial_epes = torch.randn(batch, Max_sources, num_hypothesis)

# Original implementation
spatial_epe_orig, idx_selected_orig = torch.min(spatial_epes, dim=2) 
mask_selected_orig = torch.zeros_like(spatial_epes)

for b in range(batch) :
    for s in range(Max_sources) :
        mask_selected_orig[b,s,idx_selected_orig[b,s]]=1

mask_selected_orig = 1 - mask_selected_orig

# Optimized implementation
spatial_epe_opt, idx_selected_opt = torch.min(spatial_epes, dim=2)
mask_selected_opt = torch.zeros_like(spatial_epes)
mask_selected_opt.scatter_(2, idx_selected_opt.unsqueeze(-1), 1)
mask_selected_opt = 1 - mask_selected_opt

# Check if the two implementations are equivalent
assert torch.allclose(spatial_epe_orig, spatial_epe_opt)
assert torch.allclose(idx_selected_orig, idx_selected_opt)
assert torch.allclose(mask_selected_orig, mask_selected_opt)


In [70]:
idx_selected

tensor([[7, 6, 4],
        [3, 1, 1],
        [4, 4, 7],
        [7, 9, 9],
        [5, 1, 7]])

In [66]:
import torch

# create a tensor x
x = torch.tensor([[1, 2, 3, 4], [5, 1, 7, 8], [9, 10, 1, 12]])

# find the indices of the minimum values along the second dimension
min_indices = torch.argmin(x, dim=1)

# create a mask of zeros with the same shape as x
mask = torch.zeros_like(x)

# use scatter_() to set the values at the indices of the minimum values to 0
mask.scatter_(1, min_indices.unsqueeze(1), 1)
x = x * (1 - mask)

# print the results
print(x)


TypeError: scatter_() received an invalid combination of arguments - got (src=int, dim=int, index=Tensor, ), but expected one of:
 * (int dim, Tensor index, Tensor src)
      didn't match because some of the arguments have invalid types: (dim=int, index=Tensor, !src=int!, )
 * (int dim, Tensor index, Tensor src, *, str reduce)
 * (int dim, Tensor index, Number value)
      didn't match because some of the keywords were incorrect: src
 * (int dim, Tensor index, Number value, *, str reduce)


In [63]:
idx_selected

tensor([[3, 2, 7],
        [8, 0, 7],
        [4, 2, 8],
        [0, 0, 0],
        [7, 8, 8]])

In [60]:
idx_selected

tensor([[0, 1, 5],
        [8, 4, 8],
        [8, 1, 8],
        [6, 1, 6],
        [6, 0, 0]])

In [55]:
### Tests of the draft_make_sampling_loss_ambiguous_gts function

batch = 5
Max_sources = 3
num_hypothesis = 10
hyps_stacked_t = torch.randn(batch,Max_sources,2)
source_activity_target_t = torch.randint(high=2, size=(batch,Max_sources))
direction_of_arrival_target_t = torch.randn(batch,Max_sources,2)
draft_make_sampling_loss_ambiguous_gts(hyps_stacked_t, source_activity_target_t, direction_of_arrival_target_t, 'epe-relaxed')

tensor([[17320.4922,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000, 17320.6484],
        [    0.0000, 17319.1191, 17319.1191],
        [17319.9727, 17319.9727,     0.0000]])
torch.Size([5, 3])


RuntimeError: Expected index [5, 1, 3] to be smaller than self [5, 3, 2] apart from dimension 1

In [ ]:
filling_value = torch.tensor([10000,10000])

for b in range(batch) :
    for s in range(max_sources) : 
        if source_activity_target_t[b,s] == 0 :
            direction_of_arrival_target_t[b,s,:]=filling_value

In [34]:
mask_active_sources = source_activity_target_t == 0
print(mask_active_sources)
print(mask_active_sources.shape)
mask_active_sources = mask_active_sources.unsqueeze(-1).expand_as(direction_of_arrival_target_t)
print(mask_active_sources.shape)
print(direction_of_arrival_target_t.shape)
direction_of_arrival_target_t[mask_active_sources]=1000
print(direction_of_arrival_target_t)

tensor([[False, False,  True],
        [ True, False,  True],
        [ True,  True, False],
        [False,  True, False],
        [ True, False,  True]])
torch.Size([5, 3])
torch.Size([5, 3, 2])
torch.Size([5, 3, 2])
tensor([[[-1.4688e+00,  8.8400e-01],
         [-7.7471e-01, -2.3873e+00],
         [ 1.0000e+03,  1.0000e+03]],

        [[ 1.0000e+03,  1.0000e+03],
         [-7.3306e-02,  3.8826e-01],
         [ 1.0000e+03,  1.0000e+03]],

        [[ 1.0000e+03,  1.0000e+03],
         [ 1.0000e+03,  1.0000e+03],
         [ 3.1762e-01,  4.1972e-01]],

        [[ 7.7035e-01,  9.5233e-02],
         [ 1.0000e+03,  1.0000e+03],
         [ 4.3323e-01, -4.4909e-01]],

        [[ 1.0000e+03,  1.0000e+03],
         [ 4.9183e-01,  1.7220e-01],
         [ 1.0000e+03,  1.0000e+03]]])


In [27]:
mask_active_sources.shape

torch.Size([5, 3, 2])

In [20]:
import torch
import torch.nn.functional as F

# set the batch size and maximum number of sources
batch_size = 3
max_sources = 4

# create random tensors for source_activity_target_t and direction_of_arrival_target_t
source_activity_target_t = torch.randint(high=2, size=(batch_size, max_sources))
direction_of_arrival_target_t = torch.randn(batch_size, max_sources, 2)

# create the filling_value tensor
filling_value = torch.tensor([10000, 10000])

# apply the original code
for b in range(batch_size):
    for s in range(max_sources): 
        if source_activity_target_t[b, s] == 0:
            direction_of_arrival_target_t[b, s, :] = filling_value

# apply the new code
mask = source_activity_target_t == 0
direction_of_arrival_target_t_new = direction_of_arrival_target_t.clone()
direction_of_arrival_target_t_new[mask] = filling_value

# check if the results are the same
assert torch.allclose(direction_of_arrival_target_t, direction_of_arrival_target_t_new, rtol=1e-5, atol=1e-5)


RuntimeError: Index put requires the source and destination dtypes match, got Float for the destination and Long for the source.

In [19]:
source_activity_target_t==0

tensor([[False, False,  True],
        [ True,  True, False],
        [ True, False,  True],
        [ True, False, False],
        [ True, False,  True]])